In [ ]:
%load_ext autoreload
%autoreload 2

%matplotlib widget
import matplotlib.pyplot as plt
from scipy.constants import R
import pandas as pd
import numpy as np
import os
import glob 

import ipywidgets as widgets

import pygaps as pg
from bet_class import BET

# BET analysis dashboard - BelMiniX

Written in Python using [pyGAPS](https://pygaps.readthedocs.io/en/master/)
, [Voila](https://voila.readthedocs.io/en/stable/using.html) and JupyterLab.  

Source code available on: [https://github.uio.no/SMN-Catalysis/Gas-Adsorption-Analysis](https://github.uio.no/SMN-Catalysis/Gas-Adsorption-Analysis)

by,  
Nicolai Haaber Junge  
Heterogeneous Catalysis,  
Centre for materials science and nanotechnology  
University of Oslo  
11.07.2021

### This dashboard performs BET analysis on BelMiniX .DAT measurment files.

The BET analysis uses the Rouquerol rules for selecting a pressure range. Note,
that you can also set a manual pressure range if you wish.

With this dashboard you can:
- Do BET analysis in single and batch mode
- Quickly batch process a folder of .DAT files
- Export results as individual or multiple files in excel and csv format.

---

In [ ]:
layout=widgets.Layout(width='50%', height='80px')


##### WIDGET DEFINITIONS


main_fold_lbl = widgets.Label('Select the folder above your measurment folder')
main_folder = widgets.Text(description='Main folder', layout=layout)  #  Main folder select
select_folder = widgets.Dropdown(description='Folder Select', layout=widgets.Layout(width='450px', height='30px'))
file_folder = widgets.SelectMultiple(description='File Select', layout=widgets.Layout(width='450px', height='350px'))

run_analysis = widgets.Button(description='Run BET', disabled=True)
clear_all_b = widgets.Button(description='Clear All')

manual_press_check = widgets.Checkbox(False, description='Manual pressure range')
press_range_slider = widgets.FloatRangeSlider(
    value=[1e-3, 1e-1],
    min=0,
    max=0.1,
    step=1e-3,
    description='Manual pressure range:',
    disabled=False,
    continuous_update=False,
    orientation='horizontal',
    readout=True,
    readout_format='.3f',
    style={'description_width' : 'initial'},
    layout=widgets.Layout(width='90%')
)




###### BET PLOTTING WIDGETS

isotherm_out = widgets.Output()
bet_plot = widgets.Output()
roq_plot = widgets.Output()
bet_res = widgets.Output()

plot_outputs = [isotherm_out, bet_plot, roq_plot, bet_res]

#### WIDGET EVENTS

def on_value_change(change):
    folder = os.listdir(change.new)
    select_folder.options=folder

def on_value_change_2(change):
    fold = os.path.join(main_folder.value, change.new)
    files = [_ for _ in os.listdir(fold) if _.endswith('.DAT')]
    file_folder.options = files
    run_analysis.disabled = False
    
def run_bet(b):
    file_select = os.path.join(main_folder.value, select_folder.value, file_folder.value[0])
    bet = BET(file_select)
    with bet_res:
        res = bet.BET_results 
        df = pd.DataFrame(res.values(), columns=['value'])
        df = df.set_index(pd.Index([*res]))
        display(df)
        
    with isotherm_out:
        bet.plot_isotherm()
    
    with bet_plot:
        bet.plot_bet()

    with roq_plot:
        bet.plot_roq()
        
def clear_all(b):
    for output in plot_outputs:
        output.clear_output()
    
    
main_folder.observe(on_value_change, names='value')
select_folder.observe(on_value_change_2, names='value')

run_analysis.on_click(run_bet)
clear_all_b.on_click(clear_all)

##### WIDGET LAYOUT

press = widgets.VBox([manual_press_check, press_range_slider])

a = widgets.VBox([select_folder, file_folder], layout=widgets.Layout(width='500px'))
BET_plots = widgets.HBox([bet_plot, roq_plot])

buttons = widgets.VBox([widgets.HBox([run_analysis, clear_all_b]), press], layout=widgets.Layout(width='400px'))

b = widgets.VBox([buttons, bet_res])
c = widgets.HBox(([b, BET_plots]))


d = widgets.HBox([a, c])




main = widgets.VBox([main_fold_lbl, main_folder, d, isotherm_out])    
display(main)